In [221]:
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, StackingRegressor
from sklearn.metrics import r2_score, classification_report, accuracy_score, confusion_matrix
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
from sklearn.neighbors import NearestNeighbors
from shapely import wkt
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import KBinsDiscretizer, PolynomialFeatures
from imblearn.over_sampling import SMOTE
from statsmodels.stats.outliers_influence import variance_inflation_factor
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.pipeline import make_pipeline
from sklearn.inspection import permutation_importance

## Weather

In [222]:
weather = pd.read_csv('/Users/annaywj/Desktop/SDSU/BDA600/Capstone/sd_weather_2018_2024_combined.csv')
crash = pd.read_csv('/Users/annaywj/Desktop/SDSU/BDA600/Capstone/TIMS_SD_Crashes2013-2024.csv')

In [223]:
weather['datetime'] = pd.to_datetime(weather['datetime'], errors='coerce')
crash['COLLISION_DATE'] = pd.to_datetime(crash['COLLISION_DATE'], errors='coerce')

crash_filtered = crash[(crash['COLLISION_DATE'].dt.year >= 2018) &
                          (crash['COLLISION_DATE'].dt.year <= 2024)].copy()

daily_crashes = crash_filtered.groupby('COLLISION_DATE').agg(
    TOTAL_CRASHES=('CASE_ID', 'count'),
    AVG_SEVERITY=('COLLISION_SEVERITY', 'mean')
).reset_index().rename(columns={'COLLISION_DATE': 'datetime'})

merged_weather = pd.merge(weather, daily_crashes, on='datetime', how='inner')

weather_features = ['humidity', 'cloudcover', 'windspeed', 'precip']
target_column = 'AVG_SEVERITY'

regression_ready = merged_weather[weather_features + [target_column]].dropna()

In [224]:
X = regression_ready[weather_features]
y = regression_ready[target_column]

# Add constant for OLS regression
X_ols = sm.add_constant(X)
ols_model = sm.OLS(y, X_ols).fit()

# Random Forest Regression
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
rf_r2 = r2_score(y_test, y_pred)
rf_importance = pd.Series(rf_model.feature_importances_, index=weather_features).sort_values(ascending=False)

# Output results
ols_summary = ols_model.summary()
rf_r2, rf_importance, ols_summary.tables[1]

(-0.10411692103061565,
 cloudcover    0.359832
 humidity      0.321860
 windspeed     0.269387
 precip        0.048921
 dtype: float64,
 <class 'statsmodels.iolib.table.SimpleTable'>)

## Road

In [225]:
road = pd.read_csv('/Users/annaywj/Downloads/SOC_-_Local_Roads__Speed_and_Volume_20250423.csv')

In [226]:
crash['COLLISION_DATE'] = pd.to_datetime(crash['COLLISION_DATE'], errors='coerce')
crash_filtered = crash[(crash['COLLISION_DATE'].dt.year >= 2018) & 
                       (crash['COLLISION_DATE'].dt.year <= 2024)].copy()

crash_filtered = crash_filtered.dropna(subset=['POINT_X', 'POINT_Y'])

crash_gdf = gpd.GeoDataFrame(
    crash_filtered,
    geometry=gpd.points_from_xy(crash_filtered['POINT_X'], crash_filtered['POINT_Y']),
    crs="EPSG:4326"
)

road = road.dropna(subset=['geometry'])
road_gdf = gpd.GeoDataFrame(road, geometry=gpd.GeoSeries.from_wkt(road['geometry']), crs="EPSG:4326")

crash_gdf = crash_gdf.to_crs(epsg=3857)
road_gdf = road_gdf.to_crs(epsg=3857)

road_gdf['rep_point'] = road_gdf.geometry.representative_point()
road_gdf.set_geometry('rep_point', inplace=True)

crash_coords = np.array(list(zip(crash_gdf.geometry.x, crash_gdf.geometry.y)))
road_coords = np.array(list(zip(road_gdf.geometry.x, road_gdf.geometry.y)))

nn = NearestNeighbors(n_neighbors=1, radius=50)
nn.fit(road_coords)
distances, indices = nn.kneighbors(crash_coords)

within_50m = distances[:, 0] <= 50
crash_gdf = crash_gdf[within_50m]
matched_indices = indices[within_50m].flatten()

matched_roads_clean = road_gdf.reset_index().iloc[matched_indices].reset_index(drop=True).drop(columns=['geometry'])
joined_df = pd.concat([crash_gdf.reset_index(drop=True), matched_roads_clean], axis=1)

aggregated = joined_df.groupby('osm_id').agg(
    TOTAL_CRASHES=('CASE_ID', 'count'),
    AVG_SEVERITY=('COLLISION_SEVERITY', 'mean'),
    Lanes=('Lanes', 'first'),
    Speed_Limit_MPH=('Speed Limit MPH', 'first'),
    Speed_2022_MPH=('Speed 2022 MPH', 'first'),
    Speed_Change=('1 year Speed % change', 'first'),
    AADT_Change=('1 year AADT % change', 'first')
).dropna()

X = aggregated[['Lanes', 'Speed_Limit_MPH', 'Speed_2022_MPH', 'Speed_Change', 'AADT_Change']]
y = aggregated['AVG_SEVERITY']

X_ols = sm.add_constant(X)
ols_model = sm.OLS(y, X_ols).fit()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
rf_r2 = r2_score(y_test, y_pred)
rf_importance = pd.Series(rf_model.feature_importances_, index=X.columns).sort_values(ascending=False)

ols_summary = ols_model.summary()
rf_r2, rf_importance, ols_summary.tables[1]

(-0.10429588222092101,
 AADT_Change        0.309248
 Speed_2022_MPH     0.270044
 Speed_Change       0.259344
 Lanes              0.089369
 Speed_Limit_MPH    0.071995
 dtype: float64,
 <class 'statsmodels.iolib.table.SimpleTable'>)

## Ped_party & Ped_Victim

In [227]:
ped_parties = pd.read_csv('/Users/annaywj/Downloads/Ped_Parties.csv')
ped_victims = pd.read_csv('/Users/annaywj/Downloads/Ped_Victims.csv')

In [228]:
socio_party_merged = ped_parties.merge(
    crash[['CASE_ID', 'COLLISION_SEVERITY']],
    on='CASE_ID',
    how='left'
)

socio_party_merged['AT_FAULT_BINARY'] = socio_party_merged['AT_FAULT'].map({'Y': 1, 'N': 0})

socio_party_victim_merged = socio_party_merged.merge(
    ped_victims[['CASE_ID', 'PARTY_NUMBER', 'VICTIM_AGE', 'VICTIM_DEGREE_OF_INJURY']],
    on=['CASE_ID', 'PARTY_NUMBER'],
    how='left'
)

socio_party_victim_merged = pd.get_dummies(
    socio_party_victim_merged,
    columns=['VICTIM_DEGREE_OF_INJURY'],
    prefix='INJURY',
    drop_first=True
)

model_features_victims = ['PARTY_AGE', 'VICTIM_AGE', 'AT_FAULT_BINARY'] + \
    [col for col in socio_party_victim_merged.columns if col.startswith('INJURY_')]

model_data_victims = socio_party_victim_merged[model_features_victims + ['COLLISION_SEVERITY']].dropna()

X = model_data_victims[model_features_victims]
y = model_data_victims['COLLISION_SEVERITY']

X_ols = sm.add_constant(X)
ols_model = sm.OLS(y, X_ols).fit()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
rf_r2 = r2_score(y_test, y_pred)
rf_importance = pd.Series(rf_model.feature_importances_, index=X.columns).sort_values(ascending=False)

ols_summary = ols_model.summary()
rf_r2, rf_importance, ols_summary.tables[1]

(0.6957053396275799,
 INJURY_4.0         0.436046
 INJURY_1.0         0.228706
 INJURY_2.0         0.094561
 PARTY_AGE          0.090444
 VICTIM_AGE         0.089447
 INJURY_5.0         0.030886
 INJURY_7.0         0.014858
 AT_FAULT_BINARY    0.006772
 INJURY_3.0         0.004173
 INJURY_6.0         0.004107
 dtype: float64,
 <class 'statsmodels.iolib.table.SimpleTable'>)

## Final Model

In [229]:
weather_vars = ['cloudcover', 'humidity', 'windspeed', 'precip']
weather_final = merged_weather[['datetime', 'AVG_SEVERITY'] + weather_vars].dropna()

road_vars = ['AADT_Change', 'Speed_2022_MPH', 'Speed_Change', 'Lanes']
road_final = aggregated[road_vars + ['AVG_SEVERITY']].dropna()

party_victim_vars = ['PARTY_AGE', 'VICTIM_AGE', 'INJURY_1.0', 'INJURY_2.0', 'INJURY_4.0']
victim_final = socio_party_victim_merged[party_victim_vars + ['COLLISION_SEVERITY']].dropna()

weather_final = weather_final.rename(columns={'AVG_SEVERITY': 'severity'})
road_final = road_final.rename(columns={'AVG_SEVERITY': 'severity'})
victim_final = victim_final.rename(columns={'COLLISION_SEVERITY': 'severity'})

X_weather = weather_final[weather_vars]
X_road = road_final[road_vars]
X_victim = victim_final[party_victim_vars]
y_weather = weather_final['severity']
y_road = road_final['severity']
y_victim = victim_final['severity']

def train_rf_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    rf = RandomForestRegressor(random_state=42)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    importance = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)
    return r2, importance

r2_weather, imp_weather = train_rf_model(X_weather, y_weather)
r2_road, imp_road = train_rf_model(X_road, y_road)
r2_victim, imp_victim = train_rf_model(X_victim, y_victim)

r2_weather, imp_weather, r2_road, imp_road, r2_victim, imp_victim

(-0.10146367072304785,
 cloudcover    0.360290
 humidity      0.321670
 windspeed     0.268709
 precip        0.049330
 dtype: float64,
 -0.10976422727734536,
 AADT_Change       0.337003
 Speed_2022_MPH    0.286381
 Speed_Change      0.284184
 Lanes             0.092432
 dtype: float64,
 0.648014207470236,
 INJURY_4.0    0.480172
 INJURY_1.0    0.251844
 INJURY_2.0    0.104129
 PARTY_AGE     0.084259
 VICTIM_AGE    0.079596
 dtype: float64)

In [230]:
joined_df = joined_df.rename(columns={
    'Speed 2022 MPH': 'Speed_2022_MPH',
    '1 year Speed % change': 'Speed_Change',
    '1 year AADT % change': 'AADT_Change'
})

road_features = ['AADT_Change', 'Speed_Change', 'Speed_2022_MPH', 'Lanes']
crash_road = joined_df[['CASE_ID'] + road_features]

weather_vars = ['cloudcover', 'humidity', 'windspeed', 'precip']
weather_ready = merged_weather[['datetime'] + weather_vars].dropna()
crash_weather = crash_filtered.merge(weather_ready, left_on='COLLISION_DATE', right_on='datetime', how='left')

crash_enriched = crash_weather.merge(crash_road, on='CASE_ID', how='left')

final_data = socio_party_victim_merged.merge(
    crash_enriched[['CASE_ID'] + weather_vars + road_features],
    on='CASE_ID',
    how='left'
)

selected_features = [
    'PARTY_AGE', 'VICTIM_AGE', 'INJURY_1.0', 'INJURY_2.0', 'INJURY_4.0',
    'cloudcover', 'humidity', 'windspeed', 'Lanes', 'precip',
    'AADT_Change', 'Speed_Change', 'Speed_2022_MPH', 'COLLISION_SEVERITY'
]

final_data_model = final_data[selected_features].dropna()

In [231]:
X_final = final_data_model.drop(columns='COLLISION_SEVERITY')
y_final = final_data_model['COLLISION_SEVERITY']

X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.3, random_state=42)
rf_final = RandomForestRegressor(random_state=42)
rf_final.fit(X_train, y_train)
y_pred = rf_final.predict(X_test)

r2_final = r2_score(y_test, y_pred)
feature_importance = pd.Series(rf_final.feature_importances_, index=X_final.columns).sort_values(ascending=False)

print(f"Final Model R²: {r2_final:.4f}")
print("\nFeature Importances:")
print(feature_importance)

Final Model R²: 0.7582

Feature Importances:
INJURY_4.0        0.394914
INJURY_1.0        0.201598
INJURY_2.0        0.093922
cloudcover        0.044772
Speed_2022_MPH    0.042501
humidity          0.041892
windspeed         0.037340
Speed_Change      0.035669
AADT_Change       0.034559
VICTIM_AGE        0.026779
PARTY_AGE         0.024392
Lanes             0.014395
precip            0.007266
dtype: float64


In [232]:
final_data_model = socio_party_victim_merged.merge(
    crash_enriched[['CASE_ID', 'cloudcover', 'humidity', 'windspeed', 'AADT_Change', 'Speed_Change', 'Speed_2022_MPH', 'Lanes', 'precip']],
    on='CASE_ID',
    how='left'
)

final_data_model = final_data_model[[
    'CASE_ID',  
    'PARTY_AGE', 'VICTIM_AGE', 'INJURY_1.0', 'INJURY_2.0', 'INJURY_4.0',
    'cloudcover', 'humidity', 'windspeed', 'Lanes', 'precip',
    'AADT_Change', 'Speed_Change', 'Speed_2022_MPH',
    'COLLISION_SEVERITY'
]].dropna()


In [233]:
features_updated = [col for col in final_data_model.columns if col not in ['COLLISION_SEVERITY', 'severity_log']]
X = final_data_model[features_updated].dropna()
y = final_data_model.loc[X.index, 'COLLISION_SEVERITY']

In [234]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

rf_enriched = RandomForestRegressor(random_state=42)
rf_enriched.fit(X_train, y_train)
y_pred = rf_enriched.predict(X_test)

r2_enriched = r2_score(y_test, y_pred)
importances_enriched = pd.Series(rf_enriched.feature_importances_, index=X.columns).sort_values(ascending=False)

print(f"Enriched RF Model R²: {r2_enriched:.4f}")
print("\nTop Feature Importances:")
importances_enriched

Enriched RF Model R²: 0.7720

Top Feature Importances:


INJURY_4.0        0.394914
INJURY_1.0        0.201598
INJURY_2.0        0.093922
CASE_ID           0.053137
cloudcover        0.036665
Speed_2022_MPH    0.034500
humidity          0.034384
windspeed         0.032328
Speed_Change      0.029764
AADT_Change       0.029034
VICTIM_AGE        0.023205
PARTY_AGE         0.019263
Lanes             0.011645
precip            0.005641
dtype: float64

In [235]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

X_vif = X.drop(columns=['CASE_ID'])

X_vif_const = sm.add_constant(X_vif)

vif_data = pd.DataFrame()
vif_data['Feature'] = X_vif_const.columns
vif_data['VIF'] = [variance_inflation_factor(X_vif_const.values, i) for i in range(X_vif_const.shape[1])]

vif_data.sort_values(by='VIF', ascending=False)

,Feature,VIF
0,const,73.936267
6,cloudcover,1.366702
7,humidity,1.359378
2,VICTIM_AGE,1.275914
1,PARTY_AGE,1.271574
10,precip,1.238299
8,windspeed,1.204352
13,Speed_2022_MPH,1.102314
5,INJURY_4.0,1.091316
11,AADT_Change,1.081750


In [236]:
final_data_model['severity_class'] = pd.cut(
    final_data_model['COLLISION_SEVERITY'],
    bins=[0, 1.5, 2.5, 3.5, 4.5, 5.5],
    labels=[1, 2, 3, 4, 5]
).astype(int)

X_rf = final_data_model.drop(columns=['COLLISION_SEVERITY', 'severity_class'], errors='ignore')
y_rf = final_data_model['severity_class']

X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X_rf, y_rf, test_size=0.3, random_state=42)

rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_rf, y_train_rf)
rf_preds = rf_model.predict(X_test_rf)

rf_accuracy = accuracy_score(y_test_rf, rf_preds)
rf_report = classification_report(y_test_rf, rf_preds)

print("Random Forest Classification Report (Severity Prediction)\n")
print(f"Accuracy: {rf_accuracy:.4f}\n")
print(classification_report(y_test_rf, rf_preds, digits=2))

Random Forest Classification Report (Severity Prediction)

Accuracy: 0.8574

              precision    recall  f1-score   support

           1       1.00      0.82      0.90        88
           2       0.91      0.67      0.77       194
           3       0.79      0.96      0.86       504
           4       0.93      0.83      0.88       399

    accuracy                           0.86      1185
   macro avg       0.91      0.82      0.85      1185
weighted avg       0.87      0.86      0.86      1185



## Socioeonomic

In [237]:
social_party_df = pd.read_csv("/Users/annaywj/Downloads/collisions_merged_social2018_2024.csv")
ped_victims_df = pd.read_csv("/Users/annaywj/Downloads/Ped_Victims.csv")

In [238]:
merged_with_victims = social_party_df.merge(
    ped_victims_df,
    on='CASE_ID',
    how='left'
)

In [239]:
injury_mapping = {
    '1 - Killed': 5,
    '5 - Suspected Serious Injury': 4,
    '2 - Severe Injury': 4,
    '6 - Suspected Minor Injury': 3,
    '3 - Other Visible Injury': 3,
    '7 - Possible Injury': 2,
    '4 - Complaint of Pain': 2,
    '0 - No Injury': 1,
    1: 5, 2: 4, 3: 3, 4: 2, 5: 4, 6: 3, 7: 2, 0: 1
}
merged_with_victims['INJURY_SEVERITY_NUM'] = merged_with_victims['VICTIM_DEGREE_OF_INJURY'].map(injury_mapping)

In [240]:
selected_cols = ['INJURY_SEVERITY_NUM', 'VICTIM_AGE', 'driver_PARTY_AGE', 'pedestrian_PARTY_AGE',
                 'Under 5', '5 to 9', '10 to 14', '15 to 17', '18 and 19', '20 to 24', 
                 '$15,000 to $29,999', '$30,000 to $44,999', '$45,000 to $59,999',
                 '$60,000 to $74,999', '$75,000 to $99,999', '$100,000 to $124,999',
                 '$125,000 to $149,999', '$150,000 to $199,999', '$200,000 or more',
                 'population_x']
df_corr = merged_with_victims[selected_cols].dropna()

In [241]:
features = [
    'VICTIM_AGE', 'driver_PARTY_AGE', 'pedestrian_PARTY_AGE',
    'Under 5', '5 to 9', '10 to 14', '15 to 17', '18 and 19', '20 to 24',
    '$15,000 to $29,999', '$30,000 to $44,999', '$45,000 to $59,999',
    '$60,000 to $74,999', '$75,000 to $99,999', '$100,000 to $124,999',
    '$125,000 to $149,999', '$150,000 to $199,999', '$200,000 or more',
    'population_x'
]
target = 'INJURY_SEVERITY_NUM'


In [242]:
model_df = merged_with_victims[features + [target]].dropna()

X = model_df[features]
y = model_df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, output_dict=True)

importances = pd.Series(rf.feature_importances_, index=features).sort_values(ascending=False)

In [243]:
accuracy, importances, pd.DataFrame(report)

(0.418331374853114,
 VICTIM_AGE              0.220932
 pedestrian_PARTY_AGE    0.179455
 driver_PARTY_AGE        0.157649
 $30,000 to $44,999      0.029415
 $15,000 to $29,999      0.029175
 $100,000 to $124,999    0.029129
 $150,000 to $199,999    0.028964
 $60,000 to $74,999      0.028890
 $125,000 to $149,999    0.028759
 $200,000 or more        0.028596
 $45,000 to $59,999      0.028287
 $75,000 to $99,999      0.027960
 20 to 24                0.027124
 Under 5                 0.026395
 15 to 17                0.026384
 population_x            0.026312
 18 and 19               0.026092
 10 to 14                0.025573
 5 to 9                  0.024910
 dtype: float64,
                     1           2           3           4           5  \
 precision    0.635135    0.397375    0.430057    0.259574    0.246032   
 recall       0.569697    0.424745    0.513544    0.167582    0.164021   
 f1-score     0.600639    0.410604    0.468107    0.203673    0.196825   
 support    330.00000

In [244]:
merged = social_party_df.merge(
    ped_victims_df[['CASE_ID', 'VICTIM_AGE', 'VICTIM_DEGREE_OF_INJURY']],
    on='CASE_ID', how='left'
)

merged = merged.dropna(subset=['VICTIM_DEGREE_OF_INJURY'])
merged['injury_severe'] = merged['VICTIM_DEGREE_OF_INJURY'].isin([1, 2, 5]).astype(int)

income_cols = [
    '$15,000 to $29,999', '$30,000 to $44,999', '$45,000 to $59,999',
    '$60,000 to $74,999', '$75,000 to $99,999', '$100,000 to $124,999',
    '$125,000 to $149,999', '$150,000 to $199,999', '$200,000 or more'
]
merged['low_income_pct'] = merged['$15,000 to $29,999'] / merged['population_x']
merged['mid_income_pct'] = merged['$60,000 to $74,999'] / merged['population_x']
merged['high_income_pct'] = merged['$125,000 to $149,999'] / merged['population_x']

def age_group(age):
    if pd.isna(age): return 'unknown'
    if age <= 17: return 'young'
    elif age <= 64: return 'adult'
    else: return 'senior'

merged['victim_age_group'] = merged['VICTIM_AGE'].apply(age_group)
merged = pd.get_dummies(merged, columns=['victim_age_group'], prefix='victim_age_group', drop_first=False)

for col in ['victim_age_group_young', 'victim_age_group_adult', 'victim_age_group_senior']:
    if col not in merged.columns:
        merged[col] = 0

merged['age_interaction'] = merged['VICTIM_AGE'] * merged['pedestrian_PARTY_AGE']

feature_cols = [
    'driver_PARTY_AGE', 'pedestrian_PARTY_AGE', 'VICTIM_AGE',
    'low_income_pct', 'mid_income_pct', 'high_income_pct',
    'injury_severe', 'age_interaction',
    'victim_age_group_young', 'victim_age_group_adult', 'victim_age_group_senior'
]

X = merged[feature_cols].dropna()
y = merged.loc[X.index, 'VICTIM_DEGREE_OF_INJURY'].astype(int)

sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.3, random_state=42)

rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, digits=2, output_dict=True)
importance = pd.Series(rf_model.feature_importances_, index=X.columns).sort_values(ascending=False)

accuracy, importance, pd.DataFrame(report).transpose()

(0.7042227884965416,
 injury_severe              0.146004
 low_income_pct             0.128531
 mid_income_pct             0.128427
 high_income_pct            0.127416
 driver_PARTY_AGE           0.119941
 age_interaction            0.118552
 pedestrian_PARTY_AGE       0.114377
 VICTIM_AGE                 0.102276
 victim_age_group_adult     0.006306
 victim_age_group_young     0.004924
 victim_age_group_senior    0.003246
 dtype: float64,
               precision    recall  f1-score      support
 0              0.816689  0.839662  0.828017   711.000000
 1              0.812865  0.815249  0.814056   682.000000
 2              0.818713  0.813953  0.816327   688.000000
 3              0.421222  0.385294  0.402458   680.000000
 4              0.440549  0.415230  0.427515   696.000000
 5              0.853448  0.855908  0.854676   694.000000
 6              0.676343  0.693452  0.684791   672.000000
 7              0.744536  0.812221  0.776907   671.000000
 accuracy       0.704223  0.70422

## Final Model

In [245]:
crash['COLLISION_DATE'] = pd.to_datetime(crash['COLLISION_DATE'], errors='coerce')
crash_filtered = crash[(crash['COLLISION_DATE'].dt.year >= 2018) & (crash['COLLISION_DATE'].dt.year <= 2024)].copy()
crash_filtered = crash_filtered.dropna(subset=['POINT_X', 'POINT_Y'])

crash_gdf = gpd.GeoDataFrame(crash_filtered, geometry=gpd.points_from_xy(crash_filtered['POINT_X'], crash_filtered['POINT_Y']), crs="EPSG:4326")
road = road.dropna(subset=['geometry'])
road_gdf = gpd.GeoDataFrame(road, geometry=gpd.GeoSeries.from_wkt(road['geometry']), crs="EPSG:4326")

crash_gdf = crash_gdf.to_crs(epsg=3857)
road_gdf = road_gdf.to_crs(epsg=3857)
road_gdf['rep_point'] = road_gdf.geometry.representative_point()
road_gdf.set_geometry('rep_point', inplace=True)

crash_coords = np.array(list(zip(crash_gdf.geometry.x, crash_gdf.geometry.y)))
road_coords = np.array(list(zip(road_gdf.geometry.x, road_gdf.geometry.y)))

nn = NearestNeighbors(n_neighbors=1, radius=50)
nn.fit(road_coords)
distances, indices = nn.kneighbors(crash_coords)
within_50m = distances[:, 0] <= 50
crash_gdf = crash_gdf[within_50m]
matched_indices = indices[within_50m].flatten()
matched_roads = road_gdf.reset_index().iloc[matched_indices].reset_index(drop=True).drop(columns=['geometry'])
joined_df = pd.concat([crash_gdf.reset_index(drop=True), matched_roads], axis=1)

joined_df = joined_df.rename(columns={
    'Speed 2022 MPH': 'Speed_2022_MPH',
    '1 year Speed % change': 'Speed_Change',
    '1 year AADT % change': 'AADT_Change'
})

weather['datetime'] = pd.to_datetime(weather['datetime'], errors='coerce')
daily_crashes = crash_filtered.groupby('COLLISION_DATE').agg(
    TOTAL_CRASHES=('CASE_ID', 'count'),
    AVG_SEVERITY=('COLLISION_SEVERITY', 'mean')
).reset_index().rename(columns={'COLLISION_DATE': 'datetime'})
merged_weather = pd.merge(weather, daily_crashes, on='datetime', how='inner')
weather_vars = ['cloudcover', 'humidity', 'windspeed', 'precip']
weather_ready = merged_weather[['datetime'] + weather_vars].dropna()
crash_weather = crash_filtered.merge(weather_ready, left_on='COLLISION_DATE', right_on='datetime', how='left')

road_features = ['AADT_Change', 'Speed_Change', 'Speed_2022_MPH', 'Lanes']
crash_road = joined_df[['CASE_ID'] + road_features]
crash_enriched = crash_weather.merge(crash_road, on='CASE_ID', how='left')

socio_party_merged = ped_parties.merge(crash[['CASE_ID', 'COLLISION_SEVERITY']], on='CASE_ID', how='left')
socio_party_merged['AT_FAULT_BINARY'] = socio_party_merged['AT_FAULT'].map({'Y': 1, 'N': 0})
socio_party_victim_merged = socio_party_merged.merge(
    ped_victims[['CASE_ID', 'PARTY_NUMBER', 'VICTIM_AGE', 'VICTIM_DEGREE_OF_INJURY']],
    on=['CASE_ID', 'PARTY_NUMBER'],
    how='left'
)

socio_party_victim_merged = pd.get_dummies(
    socio_party_victim_merged,
    columns=['VICTIM_DEGREE_OF_INJURY'],
    prefix='INJURY',
    drop_first=True
)

final_data_model = socio_party_victim_merged.merge(
    crash_enriched[['CASE_ID'] + weather_vars + road_features],
    on='CASE_ID',
    how='left'
)

selected_features = [
    'PARTY_AGE', 'VICTIM_AGE', 'INJURY_1.0', 'INJURY_2.0', 'INJURY_4.0',
    'cloudcover', 'humidity', 'windspeed', 'Lanes', 'precip',
    'AADT_Change', 'Speed_Change', 'Speed_2022_MPH', 'COLLISION_SEVERITY'
]
final_data_model = final_data_model[selected_features].dropna()

X = final_data_model.drop(columns='COLLISION_SEVERITY')
y = final_data_model['COLLISION_SEVERITY']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
r2_final = r2_score(y_test, y_pred)
feature_importance = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)

r2_final

0.7581738358189215

In [246]:
final_data_model = final_data_model.merge(
    crash_enriched[['CASE_ID']], left_index=True, right_index=True, how='left'
)

In [247]:
merged_unique = merged.drop_duplicates(subset='CASE_ID')

final_combined = final_data_model.merge(
    merged_unique[[
        'CASE_ID', 'driver_PARTY_AGE', 'pedestrian_PARTY_AGE', 'low_income_pct',
        'mid_income_pct', 'high_income_pct', 'injury_severe', 'age_interaction',
        'victim_age_group_young', 'victim_age_group_adult', 'victim_age_group_senior'
    ]],
    on='CASE_ID', how='inner'
)

final_features = [
    'PARTY_AGE', 'VICTIM_AGE', 'INJURY_1.0', 'INJURY_2.0', 'INJURY_4.0',
    'cloudcover', 'humidity', 'windspeed', 'Lanes', 'precip',
    'AADT_Change', 'Speed_Change', 'Speed_2022_MPH',
    'driver_PARTY_AGE', 'pedestrian_PARTY_AGE',
    'low_income_pct', 'mid_income_pct', 'high_income_pct',
    'injury_severe', 'age_interaction',
    'victim_age_group_young', 'victim_age_group_adult', 'victim_age_group_senior'
]

df_model = final_combined[final_features + ['COLLISION_SEVERITY']].dropna()
X = df_model[final_features]
y = df_model['COLLISION_SEVERITY']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

rf_final_combined = RandomForestRegressor(random_state=42)
rf_final_combined.fit(X_train, y_train)
y_pred = rf_final_combined.predict(X_test)

r2_combined = r2_score(y_test, y_pred)
importance_combined = pd.Series(rf_final_combined.feature_importances_, index=X.columns).sort_values(ascending=False)

r2_combined

0.7848755679710682

In [249]:
import statsmodels.api as sm  
X_with_const = sm.add_constant(X)

vif_df = pd.DataFrame()
vif_df["Feature"] = X_with_const.columns
vif_df["VIF"] = [variance_inflation_factor(X_with_const.values, i) for i in range(X_with_const.shape[1])]
vif_df.sort_values(by="VIF", ascending=False, inplace=True)

vif_df

,Feature,VIF
23,victim_age_group_senior,inf
22,victim_age_group_adult,inf
21,victim_age_group_young,inf
20,age_interaction,2.019231
15,pedestrian_PARTY_AGE,2.000840
7,humidity,1.452511
6,cloudcover,1.439977
10,precip,1.375808
16,low_income_pct,1.346978
8,windspeed,1.310479


In [250]:
X_interaction = X.copy()
X_interaction['humidity_x_windspeed'] = X['humidity'] * X['windspeed']
X_interaction['speed_aadt_interaction'] = X['Speed_2022_MPH'] * X['AADT_Change']
X_interaction['cloudcover_precip'] = X['cloudcover'] * X['precip']

X_train_i, X_test_i, y_train_i, y_test_i = train_test_split(X_interaction, y, test_size=0.3, random_state=42)

models = {
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42, verbosity=0),
    "Ridge": Ridge(),
    "Lasso": Lasso()
}

r2_scores = {}
for name, model in models.items():
    model.fit(X_train_i, y_train_i)
    y_pred = model.predict(X_test_i)
    r2_scores[name] = r2_score(y_test_i, y_pred)

r2_scores

{'Random Forest': 0.7916981755378338,
 'Gradient Boosting': 0.7569117978613742,
 'XGBoost': 0.7819014979717556,
 'Ridge': 0.7551451488658388,
 'Lasso': 0.013771568708790927}

In [251]:
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

rf = RandomForestRegressor(random_state=42)

grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf,
                              cv=3, n_jobs=-1, scoring='r2', verbose=1)
grid_search_rf.fit(X_train, y_train)

best_rf_model = grid_search_rf.best_estimator_
best_rf_score = grid_search_rf.best_score_
best_rf_params = grid_search_rf.best_params_

best_rf_model, best_rf_score, best_rf_params

Fitting 3 folds for each of 36 candidates, totalling 108 fits


(RandomForestRegressor(max_depth=10, random_state=42),
 0.644006359726089,
 {'max_depth': 10,
  'min_samples_leaf': 1,
  'min_samples_split': 2,
  'n_estimators': 100})

In [252]:
param_grid_rf = {
    'n_estimators': [200, 300, 500],
    'max_depth': [20, 30, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1]
}
rf = RandomForestRegressor(random_state=42)

grid_search_rf = GridSearchCV(
    estimator=rf,
    param_grid=param_grid_rf,
    cv=3,
    n_jobs=-1,
    scoring='r2',
    verbose=1
)

grid_search_rf.fit(X_train, y_train)

print("Best R²:", grid_search_rf.best_score_)
print("Best Params:", grid_search_rf.best_params_)


Fitting 3 folds for each of 18 candidates, totalling 54 fits
Best R²: 0.6399093287224126
Best Params: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}


In [253]:
X = df_model[final_features]
y = df_model['COLLISION_SEVERITY']

rf_base = RandomForestRegressor(random_state=42)
rf_base.fit(X, y)
perm_importances = permutation_importance(rf_base, X, y, n_repeats=10, random_state=42, scoring='r2')
important_features = X.columns[perm_importances.importances_mean > 0.005]  

poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_poly = poly.fit_transform(X[important_features])
poly_feature_names = poly.get_feature_names_out(important_features)

X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.3, random_state=42)

rf = RandomForestRegressor(random_state=42, n_estimators=200, max_depth=None)
xgb = XGBRegressor(random_state=42)
ridge = Ridge()
lasso = Lasso()

rf.fit(X_train, y_train)
xgb.fit(X_train, y_train)
ridge.fit(X_train, y_train)
lasso.fit(X_train, y_train)

stack = StackingRegressor(
    estimators=[('rf', rf), ('xgb', xgb), ('ridge', ridge)],
    final_estimator=Ridge(),
    passthrough=True
)
stack.fit(X_train, y_train)

results = {
    "Random Forest": r2_score(y_test, rf.predict(X_test)),
    "XGBoost": r2_score(y_test, xgb.predict(X_test)),
    "Ridge": r2_score(y_test, ridge.predict(X_test)),
    "Lasso": r2_score(y_test, lasso.predict(X_test)),
    "Stacked Ensemble": r2_score(y_test, stack.predict(X_test)),
}

results = pd.Series(results).sort_values(ascending=False)
results

Random Forest         0.769688
XGBoost               0.705062
Lasso                -0.482983
Ridge               -80.324758
Stacked Ensemble   -133.296298
dtype: float64

In [254]:
df_model = final_combined[final_features + ['COLLISION_SEVERITY']].dropna().copy()
df_model['Speed_Cloud_Interaction'] = df_model['Speed_2022_MPH'] * df_model['cloudcover']
df_model['Humidity_Wind_Interaction'] = df_model['humidity'] * df_model['windspeed']
df_model['Precip_AADT_Interaction'] = df_model['precip'] * df_model['AADT_Change']

interaction_features = final_features + [
    'Speed_Cloud_Interaction',
    'Humidity_Wind_Interaction',
    'Precip_AADT_Interaction'
]

X = df_model[interaction_features]
y = df_model['COLLISION_SEVERITY']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

rf_interaction = RandomForestRegressor(random_state=42)
rf_interaction.fit(X_train, y_train)
y_pred = rf_interaction.predict(X_test)

r2_interaction = r2_score(y_test, y_pred)
r2_interaction

0.7837791519844214

In [255]:
rf_temp = RandomForestRegressor(random_state=42)
rf_temp.fit(X, y)

feature_importances = pd.Series(rf_temp.feature_importances_, index=X.columns)
low_impact_features = feature_importances[feature_importances < 0.01].index.tolist()

X_interaction_clean = X.drop(columns=low_impact_features)

X_train_clean, X_test_clean, y_train_clean, y_test_clean = train_test_split(X_interaction_clean, y, test_size=0.3, random_state=42)
rf_clean = RandomForestRegressor(random_state=42)
rf_clean.fit(X_train_clean, y_train_clean)
y_pred_clean = rf_clean.predict(X_test_clean)
r2_clean = r2_score(y_test_clean, y_pred_clean)
print("R² after removing low-impact features:", r2_clean)

print("\nDropped features (importance < 0.01):")
print(low_impact_features)

R² after removing low-impact features: 0.7806385617581603

Dropped features (importance < 0.01):
['PARTY_AGE', 'Lanes', 'precip', 'driver_PARTY_AGE', 'injury_severe', 'victim_age_group_young', 'victim_age_group_adult', 'victim_age_group_senior', 'Precip_AADT_Interaction']


In [256]:
drop_candidates = [
    'PARTY_AGE', 'Lanes', 'precip', 'driver_PARTY_AGE', 'injury_severe',
    'victim_age_group_young', 'victim_age_group_adult', 'victim_age_group_senior',
    'Precip_AADT_Interaction'
]

X_cleaned = X_interaction.drop(columns=[col for col in drop_candidates if col in X_interaction.columns])
y_cleaned = y
X_train, X_test, y_train, y_test = train_test_split(X_cleaned, y_cleaned, test_size=0.3, random_state=42)

rf_cleaned = RandomForestRegressor(random_state=42)
rf_cleaned.fit(X_train, y_train)
y_pred = rf_cleaned.predict(X_test)

r2_cleaned = r2_score(y_test, y_pred)
print(f"Cleaned Model R²: {r2_cleaned:.4f}")

Cleaned Model R²: 0.7946
